In [ ]:
import torch.nn as nn
import torch

# AvgPool через свертку

In [ ]:
batch_size = 1
channels = 2
in_features = 4

kernel = in_features
stride = kernel
padding = 0


with torch.no_grad():
    batch = torch.rand(batch_size, channels, in_features, in_features)
    print('Input data:')
    print(batch, '\n')

    print('Shape:')
    print(batch.shape, '\n')

    weights = torch.full((channels, 1, kernel, kernel), 1 / kernel ** 2)
    
    print('Weights:')
    print(weights, '\n')

    print('Shape:')
    print(weights.shape, '\n')

    avg_pool = nn.AvgPool2d(kernel, stride = stride, padding = padding)

    conv = nn.Conv2d(channels, channels, kernel_size = kernel, bias = False, groups = channels, stride = stride, padding = padding)
    conv.weight = nn.Parameter(weights)

print('Average pooling:')
print(avg_pool(batch), '\n')

print('Shape:')
print(avg_pool(batch).shape, '\n')

print('Сonvolution:')
print(conv(batch), '\n')

print('Shape:')
print(conv(batch).shape)

# FC через свертку

In [ ]:
#batch_size = 1
channels = 1

in_features = 3
out_features = 3

with torch.no_grad():

    #batch = torch.rand(batch_size, channels, in_features)
    batch = torch.rand(channels, in_features)

    print('Input data:')
    print(batch, '\n')

    print('Shape:')
    print(batch.shape, '\n')

    fc = nn.Linear(in_features, out_features, bias = False)

    print('FC weights:')
    print(fc.weight, '\n')

    conv = nn.Conv1d(channels, channels, kernel_size = out_features * in_features, stride = in_features, padding = in_features * (out_features - 1), groups = channels, bias = False)

    weights = fc.weight.flip(0)
    weights = weights.flatten()
    weights = weights.repeat(1, channels)
    weights = weights.reshape(channels, 1, out_features * in_features)

    conv.weight = nn.Parameter(weights)

print('FC:')
print(fc(batch), '\n')

print('Сonvolution:')
print(conv(batch), '\n')